In [2]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../src_jobs/')

In [3]:
import pickle
from itertools import repeat
from pathlib import Path
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import torch
from artifact import Saw_centered
from sliding_window_detector import SlidingWindowTransformerDetector, SlidingWindowLinearDetector, FineTunedSlidingWindowDetector

from data import CachedArtifactDataset, TestArtifactDataset, CenteredArtifactDataset

In [4]:
torch.set_grad_enabled(False)

In [5]:
real_data_df = pd.read_pickle('/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/real/normalized_deviation_updated_TEST.pickle') 
ground_truth = pd.read_csv('/workspaces/AICoE_Ramping_Artefacts/artifactory-master/data/gt_changes_only_relabeled_200K.csv')

In [6]:
test_width = 512

In [7]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#autheticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient.from_config(
    credential=credential,
    path="config.json",
)

Found the config file in: config.json


In [ ]:
import azure.ai.ml._artifacts._artifact_utilities as artifact_utils

data_asset = ml_client.data.get("artifactory_output", version="1")
artifact_utils.download_artifact_from_aml_uri(uri=data_asset.path, destination="./checkpoints_transformer/", datastore_operation=ml_client.datastores)
cnn_dense_635 = SlidingWindowLinearDetector.load_from_checkpoint("/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks/checkpoints_transformer/epoch=0-step=29500.ckpt").cpu()

data_asset = ml_client.data.get("artifactory_output", version="1")
artifact_utils.download_artifact_from_aml_uri(uri=data_asset.path, destination="./checkpoints_transformer/", datastore_operation=ml_client.datastores)
cnn_dense_769 = SlidingWindowLinearDetector.load_from_checkpoint("/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks/checkpoints_transformer/epoch=0-step=49000.ckpt").cpu()

data_asset = ml_client.data.get("artifactory_output", version="1")
artifact_utils.download_artifact_from_aml_uri(uri=data_asset.path, destination="./checkpoints_transformer/", datastore_operation=ml_client.datastores)
transformer_587 = SlidingWindowTransformerDetector.load_from_checkpoint("/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks/checkpoints_transformer/epoch=0-step=28000.ckpt").cpu()

data_asset = ml_client.data.get("artifactory_output", version="1")
artifact_utils.download_artifact_from_aml_uri(uri=data_asset.path, destination="./checkpoints_transformer/", datastore_operation=ml_client.datastores)
transformer_671 = SlidingWindowTransformerDetector.load_from_checkpoint("/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks/checkpoints_transformer/epoch=0-step=30000-v2.ckpt").cpu()

data_asset = ml_client.data.get("artifactory_output", version="1")
artifact_utils.download_artifact_from_aml_uri(uri=data_asset.path, destination="./checkpoints_transformer/", datastore_operation=ml_client.datastores)
transformer_ft = FineTunedSlidingWindowDetector.load_from_checkpoint("/workspaces/AICoE_Ramping_Artefacts/artifactory-master/notebooks/checkpoints_transformer/april/epoch=0-step=18500.ckpt").cpu()


In [27]:
def baseline_detector(input: torch.Tensor) -> int:   
    input.squeeze(0)
    prediction = 0

    center = int(input.shape[1]/2)
    # flag points with very high increment as artifact
    # Calculate increments by subtracting the tensor shifted by one from the original tensor
    increments = (input[0][1:] - input[0][:-1]).abs()
    mean_increment = torch.mean(increments)
    std_increment = torch.std(increments)


    if (increments[center-1] > (mean_increment + 2*std_increment)):
        prediction = 1
    
    # # flag highest/lowest point as artifact
    # # or better also with mean/std?
    # absolute_values = input.abs()
    # if input[0][center-1] > absolute_values.sort()[-3]:
    #     prediction = 1

    return prediction

In [31]:
ground_truth = ground_truth[:800]

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_squared_error, confusion_matrix, roc_curve
import pandas as pd
import pickle

preds_cnn_1 = list()
preds_cnn_2 = list()
preds_trans_1 = list()
preds_trans_2 = list()
preds_trans_ft = list()
preds_baseline = list()
gt = list()

dist = test_width // 2

for index, row in ground_truth.iterrows():
    example_data = torch.tensor(real_data_df[0][int(row["position"]-dist) : int(row["position"]+dist)])

    prediction_cnn_1 = cnn_dense_635(example_data.unsqueeze(0))   
    prediction_cnn_2 = cnn_dense_769(example_data.unsqueeze(0))
    prediction_trans_1 = transformer_587(example_data.unsqueeze(0))
    prediction_trans_2 = transformer_671(example_data.unsqueeze(0))
    prediction_trans_ft = transformer_ft(example_data.unsqueeze(0))
    prediction_baseline = baseline_detector(example_data.unsqueeze(0))

    preds_cnn_1 = preds_cnn_1 + [prediction_cnn_1.numpy()]
    preds_cnn_2 = preds_cnn_2 + [prediction_cnn_2.numpy()]
    preds_trans_1 = preds_trans_1 + [prediction_trans_1.numpy()]
    preds_trans_2 = preds_trans_2 + [prediction_trans_2.numpy()]
    preds_trans_ft = preds_trans_ft + [prediction_trans_ft.numpy()]
    preds_baseline = preds_baseline + [prediction_baseline]
    
    gt = gt + [row["gt"]]

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(preds_trans, label="predictions", c="blue")
plt.title(f"Predictions on synthetic validation set")
plt.legend()
plt.show()

In [32]:
with open('predictions.pkl', 'rb') as file:
    preds_trans = pickle.load(file)

with open('gt.pkl', 'rb') as file:
    gt = pickle.load(file)

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, thresholds = precision_recall_curve(gt, preds_cnn_1)
J = precision + recall
ix = np.argmax(J)
best_thresh_pr_cnn1 = thresholds[ix]

plt.fill_between(recall, precision)
plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title("Precision-Recall curve CNN + Dense 635K Params")
plt.show()

# precision, recall, thresholds = precision_recall_curve(gt, preds_cnn_2)
# J = precision + recall
# ix = np.argmax(J)
# best_thresh_pr = thresholds[ix]

precision, recall, thresholds = precision_recall_curve(gt, preds_trans_1)
J = precision + recall
ix = np.argmax(J)
best_thresh_pr = thresholds[ix]

plt.fill_between(recall, precision)
plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title("Precision-Recall curve CNN + Transformer 587K Params")
plt.show()

# precision, recall, thresholds = precision_recall_curve(gt, preds_trans_2)
# J = precision + recall
# ix = np.argmax(J)
# best_thresh_pr = thresholds[ix]

precision, recall, thresholds = precision_recall_curve(gt, preds_trans_ft)
J = precision + recall
ix = np.argmax(J)
best_thresh_pr = thresholds[ix]

plt.fill_between(recall, precision)
plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title("Precision-Recall curve CNN + Transformer Fine Tuning 587K Params")
plt.show()

precision, recall, thresholds = precision_recall_curve(gt, preds_baseline)
J = precision + recall
ix = np.argmax(J)
best_thresh_pr = thresholds[ix]

plt.fill_between(recall, precision)
plt.ylabel("Precision")
plt.xlabel("Recall")
plt.title("Precision-Recall curve Baseline 0 Params")
plt.show()


In [34]:
from sklearn.metrics import fbeta_score

max_fbeta = 0

for threshold in np.linspace(0,  1,  100):
    predictions_cnn_1 = np.where(preds_cnn_1 > threshold, 1, 0)

    fbeta = fbeta_score(gt, predictions_cnn_1, average='macro', beta=0.5)

    if (fbeta > max_fbeta):
        max_fbeta = fbeta
        best_threshold_fbeta_cnn_1 = threshold

max_fbeta = 0

for threshold in np.linspace(0,  1,  100):
    predictions_cnn_2 = np.where(preds_cnn_2 > threshold, 1, 0)

    fbeta = fbeta_score(gt, predictions_cnn_2, average='macro', beta=0.5)

    if (fbeta > max_fbeta):
        max_fbeta = fbeta
        best_threshold_fbeta_cnn_2 = threshold


max_fbeta = 0

for threshold in np.linspace(0,  1,  100):
    predictions_trans_1 = np.where(preds_trans_1 > threshold, 1, 0)

    fbeta = fbeta_score(gt, predictions_trans_1, average='macro', beta=0.5)

    if (fbeta > max_fbeta):
        max_fbeta = fbeta
        best_threshold_fbeta_trans_1 = threshold

max_fbeta = 0

for threshold in np.linspace(0,  1,  100):
    predictions_trans_2 = np.where(preds_trans_2 > threshold, 1, 0)

    fbeta = fbeta_score(gt, predictions_trans_2, average='macro', beta=0.5)

    if (fbeta > max_fbeta):
        max_fbeta = fbeta
        best_threshold_fbeta_trans_2 = threshold

max_fbeta = 0

for threshold in np.linspace(0,  1,  100):
    predictions_trans_ft = np.where(preds_trans_ft > threshold, 1, 0)

    fbeta = fbeta_score(gt, predictions_trans_ft, average='macro', beta=0.5)

    if (fbeta > max_fbeta):
        max_fbeta = fbeta
        best_threshold_fbeta_ft = threshold

max_fbeta = 0


In [35]:
preds_binary_cnn_1 = [1 if value >= best_threshold_fbeta_cnn_1 else 0 for value in preds_cnn_1]
preds_binary_cnn_2 = [1 if value >= best_threshold_fbeta_cnn_2 else 0 for value in preds_cnn_2]
preds_binary_trans_1 = [1 if value >= best_threshold_fbeta_trans_1 else 0 for value in preds_trans_1]
preds_binary_trans_2 = [1 if value >= best_threshold_fbeta_trans_2 else 0 for value in preds_trans_2]
preds_binary_trans_ft = [1 if value >= best_threshold_fbeta_ft else 0 for value in preds_trans_ft]
preds_binary_baseline = preds_baseline

tn_cnn_1, fp_cnn_1, fn_cnn_1, tp_cnn_1 = confusion_matrix(gt, preds_binary_cnn_1, labels=[0, 1]).ravel()
tn_cnn_2, fp_cnn_2, fn_cnn_2, tp_cnn_2 = confusion_matrix(gt, preds_binary_cnn_2, labels=[0, 1]).ravel()
tn_trans_1, fp_trans_1, fn_trans_1, tp_trans_1 = confusion_matrix(gt, preds_binary_trans_1, labels=[0, 1]).ravel()
tn_trans_2, fp_trans_2, fn_trans_2, tp_trans_2 = confusion_matrix(gt, preds_binary_trans_2, labels=[0, 1]).ravel()
tn_trans_ft, fp_trans_ft, fn_trans_ft, tp_trans_ft = confusion_matrix(gt, preds_binary_trans_ft, labels=[0, 1]).ravel()
tn_baseline, fp_baseline, fn_baseline, tp_baseline = confusion_matrix(gt, preds_binary_baseline, labels=[0, 1]).ravel()

metrics = pd.DataFrame([{
    'detector': 'cnn_dense_635K',
    'threshold': best_threshold_fbeta_cnn_1,
    'accuracy': accuracy_score(gt, preds_binary_cnn_1),
    'precision': precision_score(gt, preds_binary_cnn_1),
    'recall': recall_score(gt, preds_binary_cnn_1),
    'mse': mean_squared_error(gt, preds_binary_cnn_1),
    'tn': tn_cnn_1,
    'fp': fp_cnn_1, 
    'fn': fn_cnn_1, 
    'tp': tp_cnn_1,
},
# {
#     'detector': 'cnn_dense_769K',
#     'threshold': best_threshold_fbeta_cnn_2,
#     'accuracy': accuracy_score(gt, preds_binary_cnn_2),
#     'precision': precision_score(gt, preds_binary_cnn_2),
#     'recall': recall_score(gt, preds_binary_cnn_2),
#     'mse': mean_squared_error(gt, preds_binary_cnn_2),
#     'tn': tn_cnn_2,
#     'fp': fp_cnn_2, 
#     'fn': fn_cnn_2, 
#     'tp': tp_cnn_2
# },
{
    'detector': 'transformer_587K',
    'threshold': best_threshold_fbeta_trans_1,
    'accuracy': accuracy_score(gt, preds_binary_trans_1),
    'precision': precision_score(gt, preds_binary_trans_1),
    'recall': recall_score(gt, preds_binary_trans_1),
    'mse': mean_squared_error(gt, preds_binary_trans_1),
    'tn': tn_trans_1,
    'fp': fp_trans_1, 
    'fn': fn_trans_1, 
    'tp': tp_trans_1
},
# {
#     'detector': 'transformer_671K',
#     'threshold': best_threshold_fbeta_trans_2,
#     'accuracy': accuracy_score(gt, preds_binary_trans_2),
#     'precision': precision_score(gt, preds_binary_trans_2),
#     'recall': recall_score(gt, preds_binary_trans_2),
#     'mse': mean_squared_error(gt, preds_binary_trans_2),
#     'tn': tn_trans_2,
#     'fp': fp_trans_2, 
#     'fn': fn_trans_2, 
#     'tp': tp_trans_2
# },    
{
    'detector': 'ft_transformer_587K',
    'threshold': best_threshold_fbeta_ft,
    'accuracy': accuracy_score(gt, preds_binary_trans_ft),
    'precision': precision_score(gt, preds_binary_trans_ft),
    'recall': recall_score(gt, preds_binary_trans_ft),
    'mse': mean_squared_error(gt, preds_binary_trans_ft),
    'tn': tn_trans_ft,
    'fp': fp_trans_ft, 
    'fn': fn_trans_ft, 
    'tp': tp_trans_ft
},
{
    'detector': 'baseline',
    'threshold': 0.5,
    'accuracy': accuracy_score(gt, preds_binary_baseline),
    'precision': precision_score(gt, preds_binary_baseline),
    'recall': recall_score(gt, preds_binary_baseline),
    'mse': mean_squared_error(gt, preds_binary_baseline),
    'tn': tn_baseline,
    'fp': fp_baseline, 
    'fn': fn_baseline, 
    'tp': tp_baseline
}])

In [36]:
metrics

,detector,threshold,accuracy,precision,recall,mse,tn,fp,fn,tp
0,cnn_dense_635K,0.929293,0.83125,0.625000,0.837438,0.16875,495,102,33,170
1,transformer_587K,0.787879,0.81375,0.600746,0.793103,0.18625,490,107,42,161
2,ft_transformer_587K,0.777778,0.82250,0.623482,0.758621,0.17750,504,93,49,154
3,baseline,0.500000,0.73875,0.125000,0.004926,0.26125,590,7,202,1


In [14]:
df = pd.DataFrame({
    'index': range(first_artifact_index, last_artifact_index, 15),
    'gt': gt,
    'predictions': preds_binary
})

In [15]:
# get indices of samples
fp_ind = (df.loc[(df['gt'] == 0) & (df['predictions'] == 1)])["index"]
fn_ind = (df.loc[(df['gt'] == 1) & (df['predictions'] == 0)])["index"]
tn_ind = (df.loc[(df['gt'] == 0) & (df['predictions'] == 0)])["index"]

In [ ]:
for index in fp_ind[:10]:
    example = torch.tensor(real_data_df[0][index-dist : index+dist])
    length  = len(example)

    prediction_trans = transformer_detector(example.unsqueeze(0))

    plt.figure(figsize=(15, 10))
    plt.plot(example, label="data", c="grey")
    plt.axvline(x=256, c="red", linestyle="--")
    plt.title(f"Predicted positive, but manually labeled negative, sample center {int(index)}")
    plt.legend()
    plt.show()